In [1]:
#librerias necesarias
import os
import pandas as pd
import numpy as np
import faiss
from faiss import write_index
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.nn.utils.rnn import pad_sequence
from config import CFG

c:\Users\pablo\anaconda3\envs\APLN\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def read_processed_data(with_na=False, n_samples=None):
    # List the files in the processed_data directory
    files = os.listdir('dataset/processed_data_2')

    # Read the files into a dataframe
    for idx, file in enumerate(files):
        if idx == 0:
            df = pd.read_csv('dataset/processed_data_2/' + file, na_values=['', ' ', 'No information found.'])
        else:
            df = pd.concat([df, pd.read_csv('dataset/processed_data_2/' + file, na_values=['', ' ', 'No information found.'])], ignore_index=True)
    
    if not with_na:
        df = df.dropna()

    if n_samples is not None:
        df = df.sample(n_samples)

    return df

df = read_processed_data(with_na = CFG.with_na, n_samples=CFG.n_samples)

In [3]:
df

,question,question_id,question_type,answer,focus,id,source,url,cui,semanticType,semanticGroup
0,What is (are) A guide to clinical trials for c...,0000001-1,information,"If you have cancer, a clinical trial may be an...",A guide to clinical trials for cancer,1,ADAM,https://www.nlm.nih.gov/medlineplus/ency/patie...,C0006826,T191,Disorders
6,What is (are) A1C test ?,0000003-1,information,A1C is a lab test that shows the average level...,A1C test,3,ADAM,https://www.nlm.nih.gov/medlineplus/ency/artic...,C0456984,T033,Disorders
8,What is (are) Aarskog syndrome ?,0000004-1,information,Aarskog syndrome is a very rare disease that a...,Aarskog syndrome,4,ADAM,https://www.nlm.nih.gov/medlineplus/ency/artic...,C0175701,T019,Disorders
9,What causes Aarskog syndrome ?,0000004-2,causes,Aarskog syndrome is a genetic disorder that is...,Aarskog syndrome,4,ADAM,https://www.nlm.nih.gov/medlineplus/ency/artic...,C0175701,T019,Disorders
10,What are the symptoms of Aarskog syndrome ?,0000004-3,symptoms,Symptoms of this condition include: Belly butt...,Aarskog syndrome,4,ADAM,https://www.nlm.nih.gov/medlineplus/ency/artic...,C0175701,T019,Disorders
...,...,...,...,...,...,...,...,...,...,...,...
46972,What is (are) Von Willebrand Disease ?,0000139-1,information,Von Willebrand disease (VWD) is a bleeding dis...,Von Willebrand Disease,139,NHLBI,http://www.nhlbi.nih.gov/health/health-topics/...,C0042974,T047,Disorders
46973,What causes Von Willebrand Disease ?,0000139-2,causes,Von Willebrand disease (VWD) is almost always ...,Von Willebrand Disease,139,NHLBI,http://www.nhlbi.nih.gov/health/health-topics/...,C0042974,T047,Disorders
46974,What are the symptoms of Von Willebrand Disease ?,0000139-3,symptoms,The signs and symptoms of von Willebrand disea...,Von Willebrand Disease,139,NHLBI,http://www.nhlbi.nih.gov/health/health-topics/...,C0042974,T047,Disorders
46975,How to diagnose Von Willebrand Disease ?,0000139-4,exams and tests,Early diagnosis of von Willebrand disease (VWD...,Von Willebrand Disease,139,NHLBI,http://www.nhlbi.nih.gov/health/health-topics/...,C0042974,T047,Disorders


In [13]:
# Ver valores unicos de focus
print([el for el in list(df['focus'].unique())])
print([el for el in list(df['semanticType'].unique())])

['A guide to clinical trials for cancer', 'A1C test', 'Aarskog syndrome', 'Aase syndrome', 'Abdominal aortic aneurysm', 'Abdominal bloating', 'Abdominal mass', 'Abdominal pain', 'Abdominal pain - children under age 12', 'Abdominal rigidity', 'Abdominal wall fat pad biopsy', 'ABO incompatibility', 'Abortion - medical', 'Abortion - surgical', 'Abortion - surgical - aftercare', 'Abscess', 'Abscess - abdomen or pelvis', 'Abscess scan - radioactive', 'Absence seizure', 'Absent pulmonary valve', 'Acanthosis nigricans', 'Acetaminophen and codeine overdose', 'Acetaminophen overdose', 'Acetone poisoning', 'Achalasia', 'Aches and pains during pregnancy', 'Achilles tendinitis', 'Achilles tendon rupture - aftercare', 'Achondrogenesis', 'Achondroplasia', 'Acidosis', 'Acne', 'Acne - self-care', 'Acoustic neuroma', 'Acoustic trauma', 'Acquired platelet function defect', 'Acrodermatitis', 'Acrodysostosis', 'Acromegaly', 'Actinic keratosis', 'Actinomycosis', 'Acute adrenal crisis', 'Acute arterial occl

In [4]:
df.keys()

Index(['question', 'question_id', 'question_type', 'answer', 'focus', 'id',
       'source', 'url', 'cui', 'semanticType', 'semanticGroup'],
      dtype='object')

In [5]:
class TextDataset(Dataset):
    def __init__(self, df):# Input is a pandas dataframe
        self.questions = df.question.tolist()
        self.question_ids = df.question_id.tolist()
        self.question_types = df.question_type.tolist()
        self.answers = df.answer.tolist()
        self.focus = df.focus.tolist()
        self.doc_id = df.id.tolist()
        self.source = df.source.tolist()
        self.url = df.url.tolist()
        self.cui = df.cui.tolist()
        self.semantic_type = df.semanticType.tolist()
        self.semantic_group = df.semanticGroup.tolist()
        
    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        return {'Q': self.questions[idx], 
                'Q_id': self.question_ids[idx], 
                'Q_T': self.question_types[idx], 
                'A': self.answers[idx],
                'F': self.focus[idx],
                'D_id': self.doc_id[idx],
                'S': self.source[idx],
                'U': self.url[idx],
                'C': self.cui[idx],
                'S_T': self.semantic_type[idx],
                'S_G': self.semantic_group[idx]}
    

def collate_fn(batch, tokenizer=AutoTokenizer.from_pretrained(CFG.embedding_model)):
    # Extrae las preguntas de los elementos del batch
    questions = [item['Q'] for item in batch]
    
    # Tokeniza las preguntas en un lote
    tokenized_questions = tokenizer(
        questions,
        return_tensors='pt',
        truncation=True,
        padding=True,
        max_length=512
    )
    
    # No hay necesidad de usar pad_sequence aquí, ya que tokenizer maneja el padding
    return {
        "input_ids": tokenized_questions['input_ids'],
        "attention_mask": tokenized_questions['attention_mask']
    }



def get_bert_embeddings(ds, batch_size=CFG.batch_size):
    dataloader = DataLoader(ds, batch_size=batch_size, shuffle=False, collate_fn=collate_fn, drop_last=False)
    model = AutoModel.from_pretrained(CFG.embedding_model)
    model = model.to(CFG.device)
    model.eval()
    embeddings = []
    with torch.no_grad():
        for batch in tqdm(dataloader):
            input_ids = batch['input_ids'].to(CFG.device)
            attention_mask = batch['attention_mask'].to(CFG.device)
            outputs = model(input_ids, attention_mask)
            last_hidden_state = outputs.last_hidden_state
            cls_embedding = last_hidden_state[:, 0, :]
            embeddings.append(cls_embedding.cpu().numpy())
    return np.concatenate(embeddings)


documents = TextDataset(df)

In [6]:
# Función para crear el índice FAISS
def create_faiss_index(embeddings):
  dimension = embeddings.shape[1]
  index = faiss.IndexFlatL2(dimension)
  index.add(embeddings)
  return index

embeddings = get_bert_embeddings(documents, CFG.batch_size)
index = create_faiss_index(embeddings)# Crea el índice FAISS con los embeddings

100%|██████████| 238/238 [00:18<00:00, 13.16it/s]


In [10]:
from torch.utils.tensorboard import SummaryWriter

# Create a SummaryWriter object
writer = SummaryWriter()

# Save the embeddings to TensorBoard
writer.add_embedding(embeddings, 
                     metadata = df.values.tolist(), 
                     metadata_header = df.columns.tolist(), 
                     tag='embeddings')

# Close the SummaryWriter
writer.close()


In [8]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [52]:
# Función para obtener los embeddings de una consulta de texto
def get_query_embedding(query_text, device = CFG.device):
    tokenizer = AutoTokenizer.from_pretrained(CFG.embedding_model)
    model = AutoModel.from_pretrained(CFG.embedding_model).to(device)
    inputs = tokenizer(query_text, return_tensors='pt', truncation=True, padding=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    query_embedding = outputs.last_hidden_state.mean(1).squeeze().cpu().numpy()
    return query_embedding

# Ejemplo de consulta
query_text = "Is ginseng good for health?"
query_embedding = get_query_embedding(query_text)
query_vector = np.expand_dims(query_embedding, axis=0)


# Realiza la búsqueda en el índice FAISS
D, I = index.search(query_vector, k=5)  # Busca los 5 documentos más similares
print("Distancias:", D)
print("Índices:", I)

def get_retrieved_info(documents, I, D):
    retrieved_info = dict()
    for i, idx in enumerate(I[0], start=1):
        retrieved_info[i] = {

            "url": documents[idx]['U'],
            "question": documents[idx]['Q'],
            "answer": documents[idx]['A'],
            "dissimilarity": D[0][i-1]
        }
    return retrieved_info

retrieved_info = get_retrieved_info(documents, I, D)
print(retrieved_info)
# retrieved_info = dict()

def format_retrieved_info(retrieved_info):
    formatted_info = "\n"
    for i, info in retrieved_info.items():
        
        formatted_info += f"Question: {info['question']}\n"
        formatted_info += f"Answer: {info['answer']}\n"
        formatted_info += f"Source: {info['url']}\n\n"
        # formatted_info += f"Dissimilarity: {info['dissimilarity']}\n\n"
    return formatted_info


formatted_info = format_retrieved_info(retrieved_info)
print(formatted_info)

# print("Consulta realizada con el texto:", query_text)
# print("Documentos más similares:")
# for i, idx in enumerate(I[0], start=1):
#     print(f"{i}: Documento {idx} con una distancia de {D[0][i-1]}")
#     print("Contenido del documento:", documents[idx]['Q'])
#     print("Respuesta obtenida:", documents[idx]['A'].replace('\n', ' ') )
#     # Devolver el answer de cada pregunta

Distancias: [[140.51773 190.76067 246.56963 259.68    263.8078 ]]
Índices: [[ 6387 10834  9103  9606  5343]]
{1: {'url': 'https://www.nlm.nih.gov/medlineplus/ency/patientinstructions/000347.htm', 'question': 'What is (are) Herbal remedies and supplements for weight loss ?', 'answer': 'You may see ads for supplements that claim to help you lose weight. But many of these claims are not true. Some of these supplements can even have serious side effects. Note for women:  Pregnant or nursing women should never take diet medicines of any kind. This includes prescription, herbal, and other over-the-counter remedies. Over-the-counter refers to medicines, herbs, or supplements you can buy without a prescription.', 'dissimilarity': 140.51773}, 2: {'url': 'https://www.nlm.nih.gov/medlineplus/ency/article/002875.htm', 'question': 'What is (are) Potato plant poisoning - green tubers and sprouts ?', 'answer': 'Potato plant poisoning occurs when someone eats the green tubers or new sprouts of the pot

# PART 2: RAG

Sacado de: https://learnbybuilding.ai/tutorials/rag-from-scratch


Aqui mas configuracion de ollama: https://github.com/jmorganca/ollama/blob/main/docs/api.md

In [55]:
import requests
import json

user_input = query_text


full_response = []


prompt = """
You are a medical sciences bot tailored for precision and succinctness. 
Your programming dictates responding directly to the user's query with utmost brevity. 
Your key task is to evaluate the user's question against your vast database of documents. 
The lower the dissimilarity between the query and the document, the more emphasis you should place on that information in your response. 
Your recommendation should be concise, backed by a URL to the most pertinent document for user reference, serving as proof of the recommendation's validity. 
Swift and relevant information retrieval is your principal function.

Given the user's question: {query_text}

And taking into account the pertinent information: 
{formatted_info}

Formulate a targeted recommendation for the user. 
The recommendation should be aligned closely with their query, and provide the source (url) of the selected info that has been provided.  
"""
prompt = prompt.format(query_text=query_text, formatted_info=formatted_info)
print(prompt)




url = 'http://localhost:11434/api/generate'
data = {
    "model": "llama2",
    "prompt": prompt
}
headers = {'Content-Type': 'application/json'}
response = requests.post(url, data=json.dumps(data), headers=headers, stream=True)

try:
    for line in response.iter_lines():
        # filter out keep-alive new lines
        if line:
            decoded_line = json.loads(line.decode('utf-8'))
            print(decoded_line['response'], end="")  # uncomment to results, token by token
            full_response.append(decoded_line['response'])
finally:
    response.close()



You are a medical sciences bot tailored for precision and succinctness. 
Your programming dictates responding directly to the user's query with utmost brevity. 
Your key task is to evaluate the user's question against your vast database of documents. 
The lower the dissimilarity between the query and the document, the more emphasis you should place on that information in your response. 
Your recommendation should be concise, backed by a URL to the most pertinent document for user reference, serving as proof of the recommendation's validity. 
Swift and relevant information retrieval is your principal function.

Given the user's question: Is ginseng good for health?

And taking into account the pertinent information: 

Question: What is (are) Herbal remedies and supplements for weight loss ?
Answer: You may see ads for supplements that claim to help you lose weight. But many of these claims are not true. Some of these supplements can even have serious side effects. Note for women:  Preg